<a href="https://colab.research.google.com/github/Brenda-Chinokoro/traffic-management-system/blob/main/TrafficManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
!pip install kaggle
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import os
from google.colab import files

print("Please upload your kaggle.json file...")
uploaded = files.upload()

Please upload your kaggle.json file...


Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
DATASET_NAME = " along2904/vehicle-detection-yolov8-dataset"  # Change this to your dataset

print(f"Downloading dataset: {DATASET_NAME}")
!kaggle datasets download -d {DATASET_NAME}

Dataset URL: https://www.kaggle.com/datasets/along2904/vehicle-detection-yolov8-dataset
License(s): apache-2.0
 92% 139M/151M [00:01<00:00, 107MB/s]
100% 151M/151M [00:01<00:00, 105MB/s]


In [21]:
!mkdir -p /content/custom_data
!unzip -q vehicle-detection-yolov8-dataset.zip -d /content/custom_data

In [22]:
!ls -la /content/custom_data

total 40
drwxr-xr-x 6 root root 4096 Jun  2 19:14 .
drwxr-xr-x 1 root root 4096 Jun  2 19:11 ..
drwxr-xr-x 2 root root 4096 Jun  2 19:11 classes
-rw-r--r-- 1 root root   24 Jun  2 19:13 classes.txt
-rw-r--r-- 1 root root  307 Jan 19  2024 data.yaml
-rw-r--r-- 1 root root  157 Jan 19  2024 README.dataset.txt
-rw-r--r-- 1 root root 1017 Jan 19  2024 README.roboflow.txt
drwxr-xr-x 4 root root 4096 Jun  2 19:14 test
drwxr-xr-x 4 root root 4096 Jun  2 19:12 train
drwxr-xr-x 4 root root 4096 Jun  2 19:12 valid


In [23]:
!mkdir -p /content/custom_data/train/images
!mkdir -p /content/custom_data/train/labels
!mkdir -p /content/custom_data/valid/images
!mkdir -p /content/custom_data/valid/labels

In [24]:
vehicle_classes = ['Car','Bus','Motorbike','Truck']

!mkdir -p /content/custom_data/classes
with open('/content/custom_data/classes.txt', 'w') as f:
    for vehicle_class in vehicle_classes:
        f.write(f"{vehicle_class}\n")

print("Created classes.txt with the following classes:")
!cat /content/custom_data/classes.txt

Created classes.txt with the following classes:
Car
Bus
Motorbike
Truck


In [27]:
import os

for root, dirs, files in os.walk("/content"):
    print(root)
    for name in dirs:
        print("  DIR:", name)
    for name in files:
        print("  FILE:", name)

/content
  DIR: .config
  DIR: custom_data
  DIR: dataset
  DIR: yolov8_training
  DIR: sample_data
  FILE: dataset.yaml
  FILE: vehicle-detection-yolov8-dataset.zip
  FILE: yolov8n.pt
  FILE: kaggle.json
/content/.config
  DIR: configurations
  DIR: logs
  FILE: .last_opt_in_prompt.yaml
  FILE: gce
  FILE: .last_update_check.json
  FILE: hidden_gcloud_config_universe_descriptor_data_cache_configs.db
  FILE: config_sentinel
  FILE: default_configs.db
  FILE: active_config
  FILE: .last_survey_prompt.yaml
/content/.config/configurations
  FILE: config_default
/content/.config/logs
  DIR: 2025.03.05
/content/.config/logs/2025.03.05
  FILE: 14.26.32.612701.log
  FILE: 14.26.22.754300.log
  FILE: 14.26.30.965194.log
  FILE: 14.26.41.645539.log
  FILE: 14.26.41.012425.log
  FILE: 14.26.00.059844.log
/content/custom_data
  DIR: train
  DIR: classes
  DIR: test
  DIR: valid
  FILE: README.dataset.txt
  FILE: data.yaml
  FILE: README.roboflow.txt
  FILE: classes.txt
/content/custom_data/train


In [30]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # Or yolov8s.pt, yolov8m.pt, etc.
model.train(
    data='/content/custom_data/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16  # adjust as needed for your GPU
)

Ultralytics 8.3.147 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/custom_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

100%|██████████| 755k/755k [00:00<00:00, 18.3MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,628 parameters, 3,011,612 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 78.8MB/s]


...

                   all        538       2771      0.926      0.941      0.975      0.751



50 epochs completed in 0.519 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.147 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:07<00:00,  2.22it/s]


                   all        538       2771      0.927      0.941      0.975      0.751
                   bus        135        174      0.937      0.947      0.978      0.791
                   car        330        768      0.935      0.951      0.984      0.793
             motorbike        420       1280      0.922      0.908      0.966      0.645
                 truck        310        549      0.913      0.956      0.973      0.777
Speed: 0.3ms preprocess, 2.4ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dcc2e1343d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [5]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Path to images folder (change to 'train/images' if you want)
img_dir = '/content/custom_data/valid/images'
img_files = sorted([os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])[:10]

for img_path in img_files:
    # Run inference
    results = model(img_path)
    # Get the rendered image with bounding boxes
    result_img = results[0].plot() if hasattr(results[0], "plot") else results[0].orig_img
    # Display
    plt.figure(figsize=(8, 6))
    plt.imshow(result_img)
    plt.title(os.path.basename(img_path))
    plt.axis('off')
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/custom_data/valid/images'

In [1]:
from google.colab import files
uploaded = files.upload()  # Choose your image file (e.g., mycar.jpg)

Saving test4.jpeg to test4.jpeg


In [4]:
results = model('test4.jpeg')
results[0].show()  # Display the image with bounding boxes

NameError: name 'model' is not defined